In [1]:
# Load all necessary packages
import sys
sys.path.append("../")
from collections import OrderedDict
import json
from pprint import pprint
from aif360.datasets import GermanDataset, BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.explainers import MetricTextExplainer, MetricJSONExplainer
from IPython.display import JSON, display_json
import pandas as pd

##### Load dataset

In [2]:
df=pd.read_csv('fairness_data.csv')
df = df.drop(columns=['Unnamed: 0','Unnamed: 0.1'])
print(df)
label_map = {1: 'Unfair', 0: 'Fair'}
protected_attribute_maps = [{1.0: 'Male', 0.0: 'Female'}]
#dataset = BinaryLabelDataset(df=df, label_names=None,protected_attribute_names=['gender'], metadata={'label_map': label_map, 'protected_attirbute_maps': protected_attribute_maps})
dataset=BinaryLabelDataset(favorable_label=0,unfavorable_label=1,df=df,label_names=['fairness'],protected_attribute_names=['gender'])



             salary  experience  gender  fairness
0      51494.981021   10.337294     1.0         0
1      27635.274358    6.078640     1.0         1
2      29303.546835    8.730298     0.0         0
3      20326.576332    3.871194     0.0         0
4      44990.474458   10.137524     1.0         0
5      39816.357168    7.279791     1.0         0
6      57830.104768   11.107974     1.0         0
7      49511.409631    8.177477     1.0         0
8      46033.406447    9.759576     1.0         0
9      43938.443721    8.679829     1.0         0
10     34863.079132    8.754441     1.0         1
11     48971.887613   10.360411     1.0         0
12     27379.154204   11.300719     0.0         0
13     39741.010227    5.853385     1.0         0
14     38838.012547    7.865720     1.0         0
15     63622.574913   14.568955     1.0         0
16     43013.654930   13.418665     0.0         0
17     35983.213434    7.095045     1.0         0
18     48759.266277   14.905346     1.0         1


##### Create metrics

In [3]:
priv = [{'gender': 1}]
unpriv = [{'gender': 0}]
bldm = BinaryLabelDatasetMetric(dataset, unprivileged_groups=unpriv, privileged_groups=priv)
#bldm_gd = BinaryLabelDatasetMetric(gd, unprivileged_groups=[{'sex': 0}], privileged_groups=[{'sex': 1}])
#print(gd)

##### Create explainers

In [4]:
text_expl = MetricTextExplainer(bldm)
json_expl = MetricJSONExplainer(bldm)

##### Text explanations

In [5]:
print(text_expl.num_positives())

Number of positive-outcome instances: 44850.0


In [6]:
print(text_expl.mean_difference())

Mean difference (mean label value on privileged instances - mean label value on unprivileged instances): -0.012249999999999983


In [7]:
print(text_expl.disparate_impact())

Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.9863805659013842


##### JSON Explanations

In [8]:
def format_json(json_str):
    return json.dumps(json.loads(json_str, object_pairs_hook=OrderedDict), indent=2)

In [9]:
print(format_json(json_expl.num_positives()))

{
  "metric": "Number Of Positives",
  "message": "Number of positive-outcome instances: 44850.0",
  "numPositives": 44850.0,
  "description": "Computed as the number of positive instances for the given (privileged or unprivileged) group.",
  "ideal": "The ideal value of this metric lies in the total number of positive instances made available"
}


In [10]:
print(format_json(json_expl.mean_difference()))

{
  "metric": "Mean Difference",
  "message": "Mean difference (mean label value on privileged instances - mean label value on unprivileged instances): -0.012249999999999983",
  "numPositivesUnprivileged": 8872.0,
  "numInstancesUnprivileged": 10000.0,
  "numPositivesPrivileged": 35978.0,
  "numInstancesPrivileged": 40000.0,
  "description": "Computed as the difference of the rate of favorable outcomes received by the unprivileged group to the privileged group.",
  "ideal": "The ideal value of this metric is 0.0"
}


In [11]:
print(format_json(json_expl.disparate_impact()))

{
  "metric": "Disparate Impact",
  "message": "Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.9863805659013842",
  "numPositivePredictionsUnprivileged": 8872.0,
  "numUnprivileged": 10000.0,
  "numPositivePredictionsPrivileged": 35978.0,
  "numPrivileged": 40000.0,
  "description": "Computed as the ratio of rate of favorable outcome for the unprivileged group to that of the privileged group.",
  "ideal": "The ideal value of this metric is 1.0 A value < 1 implies higher benefit for the privileged group and a value >1 implies a higher benefit for the unprivileged group."
}
